In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

batch_size = 8
num_epochs = 50
learning_rate = 0.0025
data_size = [100, 300, 600, 900, 1200, 1500]

normal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\normal'
abnormal_dir = 'C:\\Users\\WorkStation\\Desktop\\캡디이미지\\CAE_dataset\\abnormal'

In [3]:
normal_images = datasets.ImageFolder(
    normal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

abnormal_images = datasets.ImageFolder(
    abnormal_dir,
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

print(len(normal_images))
print(len(abnormal_images))

train_normal_images, test_normal_images = torch.utils.data.random_split(normal_images, [int(len(normal_images)*0.8), int(len(normal_images)*0.2)])
train_abnormal_images, test_abnormal_images = torch.utils.data.random_split(abnormal_images, [int(len(abnormal_images)*0.8), int(len(abnormal_images)*0.2)])

labeled_train_abnormal_images = []
for feature, index in train_abnormal_images:
    labeled_train_abnormal_images.append([feature, 1])

labeled_test_abnormal_images = []
for feature, index in test_abnormal_images:
    labeled_test_abnormal_images.append([feature, 1])

300
2100


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 5) 
        self.conv3 = nn.Conv2d(12, 24, 4)  
        self.fc1 = nn.Linear(24 * 25 * 25, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # 6 * 110 * 110
        x = self.pool(F.relu(self.conv2(x))) # 12 * 53 * 53
        x = self.pool(F.relu(self.conv3(x))) # 24 * 25 * 25
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
for number, dsize in enumerate(data_size, 0):
    
    CNN_PATH = f'CNN_model/CNN_best_model_{number}.pth'
    
    print(f"Number of abnormal data : {dsize}")
    
    train_loader = torch.utils.data.DataLoader(train_normal_images + labeled_train_abnormal_images[:dsize], batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_normal_images + labeled_test_abnormal_images[:int(dsize*0.2)], batch_size=batch_size, shuffle = True)

    print(len(train_loader))
    print(len(test_loader))
    
    cnn = CNN()
    cnn.to(device)
    
    optimizer = torch.optim.SGD(cnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = cnn(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        with torch.no_grad():
            test_loss = 0.0

            for i, (inputs, labels) in enumerate(test_loader, 0):
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = cnn(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

            if epoch % 10 == 0:
                print(f"[{epoch}/{num_epochs}] Train loss : {train_loss / len(train_loader)} Test loss : {test_loss/len(test_loader)}")

    torch.save(cnn.state_dict(), CNN_PATH)
    print("save model " + CNN_PATH ,"\n")

Number of abnormal data : 100
43
10
[0/50] Train loss : 0.6653978263222894 Test loss : 0.6714873671531677
[10/50] Train loss : 0.5689975499760273 Test loss : 0.6909897327423096
[20/50] Train loss : 0.3534376678858386 Test loss : 1.5763522386550903
[30/50] Train loss : 0.23874003769353377 Test loss : 1.771200430393219
[40/50] Train loss : 0.17107356103613627 Test loss : 1.6052270889282227
save model CNN_model/CNN_best_model_0.pth 

Number of abnormal data : 300
68
15
[0/50] Train loss : 0.6881053544142667 Test loss : 0.6940934499104817
[10/50] Train loss : 0.6791799388387624 Test loss : 0.7173091530799866
[20/50] Train loss : 0.38787815782844143 Test loss : 1.7490138173103333
[30/50] Train loss : 0.22560231446507661 Test loss : 1.8809935119003058
[40/50] Train loss : 0.16524986917262569 Test loss : 1.8143692096074422
save model CNN_model/CNN_best_model_1.pth 

Number of abnormal data : 600
105
23
[0/50] Train loss : 0.4259268543904736 Test loss : 1.36332657115291
[10/50] Train loss : 0.